In [ ]:
import sys
sys.path.append("/home/jupyter/fns/")
from data_pull_fns import *
from match_fns import *

In [ ]:
import sys
sys.path.append("/home/jupyter/fns/")
from data_pull_fns import *
from match_fns import *
from mlutils import dataset
import nltk
import pandas as pd
import numpy as np
from pyreadr import read_r
import datetime
pd.options.display.float_format = '{:.2f}'.format
from fuzzywuzzy import fuzz, process
# from pandarallel import pandarallel
# pandarallel.initialize(nb_workers=4, use_memory_fs=False, progress_bar=True)
from multiprocessing import Pool
from joblib import Parallel, delayed
pd.set_option('display.max_rows', 1000)
pd.set_option('display.max_columns', 1000)

# Data Ingestion

In [5]:
lvl = "upc"

if lvl == 'upc':
    train_filepath = "/data/sandbox/dotcom_data/train_upc_feat.csv"
    dotcom_file_path = "/data/sandbox/dotcom_data/dotcom_prod_type_feat.csv"
elif lvl == 'fineline':
    train_filepath = "/data/sandbox/dotcom_data/train_fl_feat.csv"
    dotcom_file_path = "/data/sandbox/dotcom_data/dotcom_subcat_feat.csv"
else:
    print("Please enter a valid level name!")

### Dotcom Data

In [9]:
dotcom_df = pd.read_csv(dotcom_file_path)
dotcom_df.set_index('index', inplace=True)
dotcom_df.shape

(106891, 14)

In [4]:
dotcom_df.head()

,rpt_lvl_0_id,rpt_lvl_0_nm,rpt_lvl_1_id,rpt_lvl_1_nm,rpt_lvl_2_id,rpt_lvl_2_nm,rpt_lvl_3_id,rpt_lvl_3_nm,rpt_lvl_4_id,rpt_lvl_4_nm,prod_type_nm,DEPT_NBR,FEAT,FEAT_CLEANED
index,,,,,,,,,,,,,,
0,60000.00,HOME,60200.00,74 HOME MANAGEMENT,1060278.00,WASTE HAMPERS AND HANGERS,1060288.00,WASTE,1060207.00,KITCHEN PANTRY TRASH AND RECYCLING,Garbage Cans & Wastebaskets,74,WASTE KITCHEN PANTRY TRASH AND RECYCLING Garba...,PANTRY WASTEBASKETS CANS GARBAGE WASTE TRASH K...
1,30000.00,HARDLINES,38000.00,03 STATIONERY,1038251.00,NOTETAKING BINDERS,1038253.00,NOTETAKING,1038259.00,JOURNALS,Blank Diaries & Journals,3,NOTETAKING JOURNALS Blank Diaries & Journals,NOTETAKING JOURNALS DIARIES BLANK
2,50000.00,APPAREL,56000.00,34 LADIES APPAREL,56146.00,LADIES ACTIVEWEAR ECOMM,56453.00,LADIES ACTIVE BOTTOMS ECOMM,1056464.00,LADIES ACTIVE FLEECE PANTS ECOMM,Sweatpants,34,LADIES ACTIVE BOTTOMS ECOMM LADIES ACTIVE FLEE...,SWEATPANTS LADIES BOTTOMS PANTS ACTIVE FLEECE
3,60000.00,HOME,66000.00,19 ARTS CRAFTS SEWING AND FABRICS,1066090.00,FLORAL AND NEEDLECRAFT,1066002.00,ARTIFICIAL PLANTS AND FLORAL,1066018.00,ARTIFICIAL TREES,Artificial Trees,19,ARTIFICIAL PLANTS AND FLORAL ARTIFICIAL TREES ...,ARTIFICIAL FLORAL PLANTS TREES
4,20000.00,ENTERTAINMENT TOYS AND SEASONAL,25000.00,07 TOYS,1025142.00,BIKES,1025148.00,BIKE ACCESSORIES,1034240.00,BIKE REPAIR AND COMPONENTS,Bicycle Derailleurs,7,BIKE ACCESSORIES BIKE REPAIR AND COMPONENTS Bi...,DERAILLEURS ACCESSORIES COMPONENTS REPAIR BICY...


Store Data 

In [5]:
train = pd.read_csv(train_filepath)
train.set_index('index', inplace=True)
train.shape

(1625736, 16)

In [6]:
train.head()

,SBU,SP_CLASS,DEPT_NBR,DEPT_DESC,CATG_NBR,CATG_DESC,SUBCATG_NBR,SUBCATG_DESC,FINELINE_NBR,FINELINE_DESC,UPC_NBR,UPC_DESC,REVISED_CATEGORY,REVISED_VARIETY,FEAT,FEAT_CLEANED
index,,,,,,,,,,,,,,,,
0,APPAREL,INTIMATES,29,INTIMATE APPAREL,101,LADY YOUNG ADULT UNDERWEAR,262,LADY YOUNG ADULT PACKAGED UNDERWEAR,1307,NB P6 PROMO BTS,19454606303,6 PK BIKINI,UNDERWEAR,UNDERWEAR,LADY YOUNG ADULT UNDERWEAR LADY YOUNG ADULT PA...,YOUNG NB BIKINI ADULT UNDERWEAR 6 P6 PK LADY B...
1,APPAREL,INTIMATES,29,INTIMATE APPAREL,101,LADY YOUNG ADULT UNDERWEAR,262,LADY YOUNG ADULT PACKAGED UNDERWEAR,1307,NB P6 PROMO BTS,19454606304,6 PK BIKINI,UNDERWEAR,UNDERWEAR,LADY YOUNG ADULT UNDERWEAR LADY YOUNG ADULT PA...,YOUNG NB BIKINI ADULT UNDERWEAR 6 P6 PK LADY B...
2,APPAREL,INTIMATES,29,INTIMATE APPAREL,101,LADY YOUNG ADULT UNDERWEAR,262,LADY YOUNG ADULT PACKAGED UNDERWEAR,1307,NB P6 PROMO BTS,19454606305,6 PK BIKINI,UNDERWEAR,UNDERWEAR,LADY YOUNG ADULT UNDERWEAR LADY YOUNG ADULT PA...,YOUNG NB BIKINI ADULT UNDERWEAR 6 P6 PK LADY B...
3,APPAREL,INTIMATES,29,INTIMATE APPAREL,101,LADY YOUNG ADULT UNDERWEAR,262,LADY YOUNG ADULT PACKAGED UNDERWEAR,1307,NB P6 PROMO BTS,19454606306,6 PK BIKINI,UNDERWEAR,UNDERWEAR,LADY YOUNG ADULT UNDERWEAR LADY YOUNG ADULT PA...,YOUNG NB BIKINI ADULT UNDERWEAR 6 P6 PK LADY B...
4,APPAREL,INTIMATES,29,INTIMATE APPAREL,101,LADY YOUNG ADULT UNDERWEAR,262,LADY YOUNG ADULT PACKAGED UNDERWEAR,1307,NB P6 PROMO BTS,19454606307,6 PK BIKINI,UNDERWEAR,UNDERWEAR,LADY YOUNG ADULT UNDERWEAR LADY YOUNG ADULT PA...,YOUNG NB BIKINI ADULT UNDERWEAR 6 P6 PK LADY B...


In [3]:
sbu_sp_class_map = pd.read_csv("/data/sandbox/dotcom_data/sbu_sp_class_map_dotcom.csv")
sbu_sp_class_map.head()

,rpt_lvl_0_nm,rpt_lvl_1_nm,DEPT_NBR,COUNTRY_CODE,SBU,SP_CLASS,DEPT_DESC
0,FOOD,01 CANDY AND TOBACCO,1,US,FOOD,CANDY AND IMPULSE,CANDY AND TOBACCO
1,CONSUMABLES,02 PERSONAL CARE,2,US,CONSUMABLES,HBA,PERSONAL CARE
2,HARDLINES,03 STATIONERY,3,US,HARDLINES,STATIONERY,STATIONERY
3,CONSUMABLES,04 HOUSEHOLD PAPER,4,US,CONSUMABLES,PAPER & PETS,HOUSEHOLD PAPER GOODS
4,ENTERTAINMENT TOYS AND SEASONAL,05 MEDIA AND GAMING,5,US,ENTERTAINMENT TOYS AND SEASONAL,GAMES & ELECTRONICS,MEDIA & GAMING


## Fuzzy matching for every SBU-DEPT 

In [11]:
import multiprocessing as mp
pool = mp.Pool(mp.cpu_count())

In [ ]:
%%time
master_df_list = []
ratio_type= 'sort'
partial=False
for row in list(sbu_sp_class_map.iterrows()):
    
    
    try:
        print("Row number of the SBU DEPT map : {0}".format(row[0]))
        print("============= SBU : {0} & DEPT : {1} ================".format(row[1]['rpt_lvl_0_nm'], row[1]['rpt_lvl_1_nm']))
    #     Get the filtered dotcom data as per the SBU and DEPT in sbu_dept_map
        dotcom_filter = (dotcom_df['rpt_lvl_0_nm'] == row[1]['rpt_lvl_0_nm']) & (dotcom_df['DEPT_NBR'] == row[1]['DEPT_NBR'])
        dotcom = dotcom_df.loc[dotcom_filter, :]
#         print(dotcom.FEAT_CLEANED.drop_duplicates().shape)
        print(dotcom.shape)
        print("Number of items in SBU {0} and DEPARTMENT  {1}: {2}".format(row[1]['rpt_lvl_0_nm'], row[1]['rpt_lvl_1_nm'], dotcom.shape[0]))

    #     Get the filtered Item data as per the SBU and DEPT mapping in sbu_dept_map

        train_filter = (train['SBU'] == row[1]['SBU']) & (train['DEPT_NBR'] == row[1]['DEPT_NBR'])
        store = train.loc[train_filter, :]
#         print(store.FEAT_CLEANED.drop_duplicates().shape)
        print(store.shape)
        print("Number of items in SBU: {0} and DEPT: {1} in store data: {2}".format(row[1]['SBU'], row[1]['DEPT_DESC'], store.shape[0]))
        
        matched_dict_list = pool.starmap(fn_fuzzy_match_store, [(x, store['FEAT_CLEANED'],ratio_type, partial,3) for x in dotcom.FEAT_CLEANED.items()])

#         matched_dict_list = []
#         for idx, value in dotcom['FEAT_CLEANED'].items():
#             matched_dict = fn_fuzzy_match_store(idx,
#                                                 value, 
#                                                 store['FEAT_CLEANED'], 
#                                                 ratio_type=ratio_type, 
#                                                 partial = partial, 
#                                                 top_n = 3) # this returns a dictionary of matches
            
# # #             Append all the dictionaries to a list 
#             matched_dict_list.append(matched_dict) # list of match dictionaries

        combined_df = pd.DataFrame(matched_dict_list)
#         combined_df['rpt_lvl_0_nm'] = row[1]['rpt_lvl_0_nm']
#         combined_df['rpt_lvl_1_nm'] = row[1]['rpt_lvl_1_nm']
        
#         matched_df_dotcom = pd.merge(combined_df, dotcom, how = "inner",left_on = 'FEAT1', right_on='FEAT_CLEANED')
#         overall_merged = pd.merge(matched_df_dotcom, store, how = "inner", left_on = f'Match1_{ratio_type}', right_on = 'FEAT_CLEANED', suffixes = ("_dotcom", "_Store"))
#         print("Number of rows in merged dotcom dataset: {}".format(overall_merged.shape[0]))
        master_df_list.append(combined_df)
    
        
        
    except:
        print("Failed for row {}".format(row[0]))
        
        
    
    print("\n")
pool.close()

Row number of the SBU DEPT map : 0
============= SBU : FOOD & DEPT : 01 CANDY AND TOBACCO ================
(862, 14)
Number of items in SBU FOOD and DEPARTMENT  01 CANDY AND TOBACCO: 862
(7836, 16)
Number of items in SBU: FOOD and DEPT: CANDY AND TOBACCO in store data: 7836


Row number of the SBU DEPT map : 1
============= SBU : CONSUMABLES & DEPT : 02 PERSONAL CARE ================
(1154, 14)
Number of items in SBU CONSUMABLES and DEPARTMENT  02 PERSONAL CARE: 1154
(14191, 16)
Number of items in SBU: CONSUMABLES and DEPT: PERSONAL CARE in store data: 14191


Row number of the SBU DEPT map : 2
============= SBU : HARDLINES & DEPT : 03 STATIONERY ================
(1946, 14)
Number of items in SBU HARDLINES and DEPARTMENT  03 STATIONERY: 1946
(13845, 16)
Number of items in SBU: HARDLINES and DEPT: STATIONERY in store data: 13845


Row number of the SBU DEPT map : 3
============= SBU : CONSUMABLES & DEPT : 04 HOUSEHOLD PAPER ================
(139, 14)
Number of items in SBU CONSUMABLES a

In [ ]:
master_df = pd.concat(master_df_list, axis = "rows").reset_index(drop=True)

In [ ]:
master_df.shape

In [ ]:
master_df['best_match_list'] = master_df.apply(lambda row: fn_best_match(row,ratio_type_prev=ratio_type, ratio_type_new='set', partial=False), axis=1)
master_df['best_match'] = master_df.best_match_list.apply(lambda x: x[0])
master_df['best_match_index'] = master_df.best_match_list.apply(lambda x: x[1])
master_df['best_match_score'] = master_df.best_match_list.apply(lambda x: x[2])

In [ ]:
master_df.drop('best_match_list', axis = 'columns', inplace=True)

In [ ]:
master_df.set_index('dc_index', inplace=True)

In [ ]:
merged_dotcom = master_df.join(dotcom_df, how = "left")

In [ ]:
# merged_dotcom.head(100)

In [ ]:
merged_dotcom.set_index('best_match_index', inplace=True)

In [ ]:
merged_final = merged_dotcom.join(train, how = "left", lsuffix='_dotcom', rsuffix='_store')

In [ ]:
merged_final.head()

In [ ]:
# merged_final.loc[:, ['FEAT_dotcom', 'FEAT_store']].head(100)

In [ ]:
merged_final.shape

In [ ]:
t = datetime.datetime.now()
filename = "fuzzywuzzy_matches_dotcom_items_lvl_{0}_({1}).csv".format(lvl, t)
print(filename)

In [ ]:
merged_final.to_csv(f"/data/sandbox/dotcom_data/{filename}")

In [ ]:
# Parallel Processing with multiprocessing library

In [ ]:
# import multiprocessing as mp
# print("Number of processors: ", mp.cpu_count())

# import numpy as np
# from time import time

# # Prepare data
# np.random.RandomState(100)
# arr = np.random.randint(0, 10, size=[200000, 5])
# data = arr.tolist()
# data[:5]

# def howmany_within_range(row, minimum, maximum):
#     """Returns how many numbers lie within `maximum` and `minimum` in a given `row`"""
#     count = 0
#     for n in row:
#         if minimum <= n <= maximum:
#             count = count + 1
#     return count


# %%time
# results = []
# for row in data:
#     results.append(howmany_within_range(row, minimum=4, maximum=8))

# print(results[:10])

# import multiprocessing as mp

# # Step 1: Init multiprocessing.Pool()
# pool = mp.Pool(32)

# %%time
# # Step 2: `pool.apply` the `howmany_within_range()`
# results_async = [pool.apply_async(howmany_within_range, args=(row, 4, 8)) for row in data]
# results = [r.get() for r in results_async] 
# # Step 3: Don't forget to close
# pool.close()    

# print(results[:10])

# # Parallelizing using Pool.apply()

# import multiprocessing as mp

# # Step 1: Init multiprocessing.Pool()
# pool = mp.Pool(mp.cpu_count())

# %%time
# # Step 2: `pool.apply` the `howmany_within_range()`
# results = [pool.apply(howmany_within_range, args=(row, 4, 8)) for row in data]

# # Step 3: Don't forget to close
# pool.close()    

# print(results[:10])
# #> [3, 1, 4, 4, 4, 2, 1, 1, 3, 3]

# # Parallelizing using Pool.map()
# import multiprocessing as mp

# # Redefine, with only 1 mandatory argument.
# def howmany_within_range_rowonly(row, minimum=4, maximum=8):
#     count = 0
#     for n in row:
#         if minimum <= n <= maximum:
#             count = count + 1
#     return count

# pool = mp.Pool(mp.cpu_count())

# results = pool.map(howmany_within_range_rowonly, [row for row in data])

# pool.close()

# print(results[:10])
# #> [3, 1, 4, 4, 4, 2, 1, 1, 3, 3]

# # Parallelizing with Pool.starmap()
# import multiprocessing as mp

# pool = mp.Pool(mp.cpu_count())

# %%time
# results = pool.starmap(howmany_within_range, [(row, 4, 8) for i,row in enumerate(data)])

# pool.close()

# print(results[:10])

# store['FEAT_CLEANED'].shape

# dotcom['FEAT_CLEANED'].items()

# list(dotcom_df['FEAT_CLEANED'].items())

# def convert_to_lower(i, x,y,z):
#     return x[y:z], i

# import multiprocessing as mp

# pool = mp.Pool(mp.cpu_count())

# m = pool.starmap(convert_to_lower, [(idx, value, 4,9) for idx,value in dotcom.FEAT_CLEANED.items()])
# pool.close()

# matched_dict_list = [pool.apply_async(fn_fuzzy_match_store, args=(idx, value, store['FEAT_CLEANED'], ratio_type,partial,3)) for idx, value in dotcom['FEAT_CLEANED'].items()]
#         matched_dict_list = [r.get() for r in matched_dict_list]



# sbu_sp_class_map

# food_dotcom = dotcom_df[dotcom_df['rpt_lvl_0_nm'].isin(['FOOD', 'APPAREL'])]

# food_store = train_fl[train_fl['SBU'].isin(['FOOD', 'APPAREL']) ]

# dotcom = food_dotcom['FEAT_CLEANED']
# store = food_store['FEAT_CLEANED']

# food_store.shape

# food_dotcom.shape

# %%time
# matched_dict_list = pool.starmap(fn_fuzzy_match_store, [(idx, value, food_store['FEAT_CLEANED'],ratio_type, partial,3) for idx,value in food_dotcom.FEAT_CLEANED.items()])

# pool.close()

# matched_dict_list

# from multiprocessing import  Pool
# pool = Pool(processes=5) 
# result_async = [pool.apply_async(get_syn_after_filtering, args = (w,60)) for w in unique_words]
# results = [r.get() for r in result_async] 

In [2]:
import pandas as pd
df = pd.read_csv("/data/sandbox/dotcom_data/fuzzywuzzy_matches_dotcom_items_lvl_upc_(2021-03-16 13:15:23.261480).csv")

In [3]:
df.shape

(105733, 43)

In [5]:
df.head(100)

,Unnamed: 0,FEAT1,Match1_sort,Match2_sort,Match3_sort,index_1_sort,index_2_sort,index_3_sort,score_1_sort,score_2_sort,score_3_sort,best_match,best_match_score,rpt_lvl_0_id,rpt_lvl_0_nm,rpt_lvl_1_id,rpt_lvl_1_nm,rpt_lvl_2_id,rpt_lvl_2_nm,rpt_lvl_3_id,rpt_lvl_3_nm,rpt_lvl_4_id,rpt_lvl_4_nm,prod_type_nm,DEPT_NBR_dotcom,FEAT_dotcom,FEAT_CLEANED_dotcom,SBU,SP_CLASS,DEPT_NBR_store,DEPT_DESC,CATG_NBR,CATG_DESC,SUBCATG_NBR,SUBCATG_DESC,FINELINE_NBR,FINELINE_DESC,UPC_NBR,UPC_DESC,REVISED_CATEGORY,REVISED_VARIETY,FEAT_store,FEAT_CLEANED_store
0,60,YOUNG ADULT LADIES UNDERWEAR UNDERPANTS PACKAGED,YOUNG NB PANTYHOSE ADULT UNDERWEAR 5PR LADY BO...,YOUNG NB PANTYHOSE ADULT UNDERWEAR 5PR LADY BO...,YOUNG NB PANTYHOSE ADULT UNDERWEAR 5PR LADY BO...,60,61,62,77,77,77,YOUNG NB PANTYHOSE ADULT UNDERWEAR 5PR LADY BO...,77,50000.00,APPAREL,50400.00,29 INTIMATE APPAREL,1050662.00,LADIES UNDERWEAR ECOMM,1050687.00,LADIES YOUNG ADULT UNDERWEAR ECOMM,1050689.00,LADIES YOUNG ADULT PACKAGED UNDERWEAR ECOMM,Underpants,29,LADIES YOUNG ADULT UNDERWEAR ECOMM LADIES YOUN...,YOUNG ADULT LADIES UNDERWEAR UNDERPANTS PACKAGED,APPAREL,INTIMATES,29,INTIMATE APPAREL,101,LADY YOUNG ADULT UNDERWEAR,262,LADY YOUNG ADULT PACKAGED UNDERWEAR,6040,NB 5PR BOX,88561566265,PANTYHOSE,UNDERWEAR,UNDERWEAR,LADY YOUNG ADULT UNDERWEAR LADY YOUNG ADULT PA...,YOUNG NB PANTYHOSE ADULT UNDERWEAR 5PR LADY BO...
1,60,YOUNG ADULT LADIES UNDERWEAR PANTIES PACKAGED,YOUNG NB PANTYHOSE ADULT UNDERWEAR 5PR LADY BO...,YOUNG NB PANTYHOSE ADULT UNDERWEAR 5PR LADY BO...,YOUNG NB PANTYHOSE ADULT UNDERWEAR 5PR LADY BO...,60,61,62,79,79,79,YOUNG NB PANTYHOSE ADULT UNDERWEAR 5PR LADY BO...,79,50000.00,APPAREL,50400.00,29 INTIMATE APPAREL,1050711.00,LADIES UNDERWEAR,1050738.00,LADIES YOUNG ADULT UNDERWEAR,1050741.00,LADIES YOUNG ADULT PACKAGED UNDERWEAR,Panties,29,LADIES YOUNG ADULT UNDERWEAR LADIES YOUNG ADUL...,YOUNG ADULT LADIES UNDERWEAR PANTIES PACKAGED,APPAREL,INTIMATES,29,INTIMATE APPAREL,101,LADY YOUNG ADULT UNDERWEAR,262,LADY YOUNG ADULT PACKAGED UNDERWEAR,6040,NB 5PR BOX,88561566265,PANTYHOSE,UNDERWEAR,UNDERWEAR,LADY YOUNG ADULT UNDERWEAR LADY YOUNG ADULT PA...,YOUNG NB PANTYHOSE ADULT UNDERWEAR 5PR LADY BO...
2,60,YOUNG ADULT LADIES UNDERWEAR PANTIES PACKAGED,YOUNG NB PANTYHOSE ADULT UNDERWEAR 5PR LADY BO...,YOUNG NB PANTYHOSE ADULT UNDERWEAR 5PR LADY BO...,YOUNG NB PANTYHOSE ADULT UNDERWEAR 5PR LADY BO...,60,61,62,79,79,79,YOUNG NB PANTYHOSE ADULT UNDERWEAR 5PR LADY BO...,79,50000.00,APPAREL,50400.00,29 INTIMATE APPAREL,1050662.00,LADIES UNDERWEAR ECOMM,1050687.00,LADIES YOUNG ADULT UNDERWEAR ECOMM,1050689.00,LADIES YOUNG ADULT PACKAGED UNDERWEAR ECOMM,Panties,29,LADIES YOUNG ADULT UNDERWEAR ECOMM LADIES YOUN...,YOUNG ADULT LADIES UNDERWEAR PANTIES PACKAGED,APPAREL,INTIMATES,29,INTIMATE APPAREL,101,LADY YOUNG ADULT UNDERWEAR,262,LADY YOUNG ADULT PACKAGED UNDERWEAR,6040,NB 5PR BOX,88561566265,PANTYHOSE,UNDERWEAR,UNDERWEAR,LADY YOUNG ADULT UNDERWEAR LADY YOUNG ADULT PA...,YOUNG NB PANTYHOSE ADULT UNDERWEAR 5PR LADY BO...
3,4148,YOUNG ADULT LADIES UNDERWEAR UNDERPANTS OPEN PACK,YOUNG PANTYHOSE ADULT PACK UNDERWEAR OPEN LADY...,YOUNG PANTYHOSE ADULT PACK UNDERWEAR OPEN NEW ...,YOUNG PANTYHOSE ADULT PACK UNDERWEAR OPEN NEW ...,4148,3974,3975,85,81,81,YOUNG PANTYHOSE ADULT PACK UNDERWEAR OPEN LADY...,85,50000.00,APPAREL,50400.00,29 INTIMATE APPAREL,1050662.00,LADIES UNDERWEAR ECOMM,1050687.00,LADIES YOUNG ADULT UNDERWEAR ECOMM,1050688.00,LADIES YOUNG ADULT OPEN PACK UNDERWEAR ECOMM,Underpants,29,LADIES YOUNG ADULT UNDERWEAR ECOMM LADIES YOUN...,YOUNG ADULT LADIES UNDERWEAR UNDERPANTS OPEN PACK,APPAREL,INTIMATES,29,INTIMATE APPAREL,101,LADY YOUNG ADULT UNDERWEAR,263,LADY YOUNG ADULT OPEN PACK UNDERWEAR,6071,LICENSE PANTYHOSE,76876211226,PANTYHOSE,UNDERWEAR,HANGING PANTY,LADY YOUNG ADULT UNDERWEAR LADY YOUNG ADULT OP...,YOUNG PANTYHOSE ADULT PACK UNDERWEAR OPEN LADY...
4,4148,YOUNG ADULT LADIES UNDERWEAR PANTIES OPEN PACK,YOUNG PANTYHOSE ADULT PACK UNDERWEAR OPEN LADY...,YOUNG PANTYHOSE ADULT PAC